In [23]:
import vk_api
import json
from pandas.io.json import json_normalize
import pandas as pd
import datetime

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import time


Задача предсказать возраст

метрика MSE

In [2]:
vk_session = vk_api.VkApi('', '')
vk_session.auth()



In [3]:
vk_api = vk_session.get_api()

In [155]:
#posts = getLikes (5117073,1, vk_api)

In [4]:
def get_friends(userId, api):
    user_friends = api.friends.get(user_id=userId, order='hints', fields = 'id, nickname, domain, sex, bdate, city, country, timezone, has_mobile, contacts, education, online, relation,  status, universities , faculty, faculty_name, relation') # возвращает в том порядке, как расположены в разделе Мои
    return user_friends

In [39]:
def get_posts(userId, api):
    user_posts = api.wall.get(owner_id=userId, filter='owner', count = 100) 
    return user_posts

In [42]:
def transformDataToDf(data, file_txt, file_csv):
    with open(file_txt,'w', encoding='utf-8') as file:
    json.dump(posts,file, sort_keys = True, indent = 4, ensure_ascii=False)
        
    df = pd.read_json(file_txt, encoding='utf-8')
    df2 = json_normalize(df['items'])
    df2.to_csv(file_csv,sep = ',')
    
    df = pd.read_csv(file_csv,  sep=',')
    
    return df

### Получение данных по друзьям

In [126]:
friends =  get_friends(5117073,vk_api)

In [127]:
df_f = transformDataToDf(friends, 'c:/friends.txt', 'c:/friends.csv')

In [128]:
df_f = df_f.drop(df_f[df_f['deactivated']=='deleted'].index)
df_f = df_f.drop(df_f[df_f['deactivated'] =='banned'].index)
df_f.shape

(122, 35)

In [129]:
#Определяем возраст человека, если не указан ставим 0
df_f['bdate'] = df_f['bdate'].astype('str')
df_f['bdate']= df_f['bdate'].apply(lambda x: '01.01.2019' if len(x) < 8 else x)
df_f['age'] = df_f.bdate.str[-4:]
df_f['age'] = df_f['age'].astype('int')
tday = datetime.datetime.now()
df_f['age2'] = tday.year
df_f['age2'].astype('int')
df_f['age3'] = df_f['age2'] - df_f['age']




In [130]:
cols = ['id','first_name','last_name','graduation', 'city.id','city.title','country.id','country.title','education_form','education_status','faculty','faculty_name','relation','relation_partner.id',
'sex','status','bdate','age3']
df_f = df_f[cols]

In [131]:
df_f[df_f['id']==6194676]

,id,first_name,last_name,graduation,city.id,city.title,country.id,country.title,education_form,education_status,faculty,faculty_name,relation,relation_partner.id,sex,status,bdate,age3


In [133]:
df_f.head()

,id,first_name,last_name,graduation,city.id,city.title,country.id,country.title,education_form,education_status,faculty,faculty_name,relation,relation_partner.id,sex,status,bdate,age3
0,88111,Анастасия,Ахметзянова,0.0,2.0,Санкт-Петербург,1.0,Россия,NaN,NaN,0.0,NaN,3.0,NaN,1,All you need is love,01.01.2019,0
1,3567843,Константин,Стрельцов,2004.0,284.0,Караганда,4.0,Казахстан,Очное отделение,NaN,9918.0,Факультет инновационных технологий (Информацио...,4.0,NaN,2,NaN,19.7.1982,37
2,9067562,Дарья,Баскакова,0.0,99.0,Новосибирск,1.0,Россия,NaN,NaN,0.0,NaN,4.0,1663884.0,1,NaN,01.01.2019,0
3,7862738,Ольга,Стрельцова,0.0,284.0,Караганда,4.0,Казахстан,NaN,NaN,0.0,NaN,4.0,NaN,1,Happiness is the truth,11.1.1983,36
4,202788920,Margo,Almatyfashion,0.0,183.0,Алматы,4.0,Казахстан,NaN,NaN,0.0,NaN,0.0,NaN,1,Тел 87078143309,01.01.2019,0


### Получение данных - посты
собираем в цикле посты каждого друга

In [106]:
users_list = df_f.id


In [18]:
#df = df[df['text'].notnull()]
#cols = ['owner_id','text','comments.count','date','post_source.platform','likes.count','reposts.count']
#df = df[cols]

In [81]:
posts =  get_posts(88111,vk_api)

In [ ]:
#
df_all = pd.DataFrame()
for user_id in users_list:
    posts =  get_posts(user_id,vk_api)
    df = transformDataToDf(posts, 'c:/data.json', 'c:/data.csv')
    if df.shape[0] > 0  :
        df = df[df['text'].notnull()]
        df['text'] = df['text'].astype('str')
        df = df[df['text'].str.len()>100]
        cols = ['owner_id','text','comments.count','date','likes.count','reposts.count']
        df = df[cols]
    
        df_all = df_all.append(df)
        

In [121]:
#выгружаем в файл для повторного использования без обращения к вк
df_all.to_csv('c:/data_all.csv',sep = ',', encoding='utf-8')

In [122]:
#считаваем из файла, чтобы не обращатья повторно к вк (после первого прохода, запускаем только получение из файла)
df_a = pd.read_csv('c:/data_all.csv',  sep=',')

In [134]:
#объединяем данные с постов с информацией о пользователе
df_u = pd.merge(df_a,df_f, how = 'inner',left_on="owner_id" , right_on="id" )

In [135]:
df_val = df_u[df_u['age3']==0] #убираем 
df_u = df_u[df_u['age3']>0]





In [150]:
df_u.shape, df_f.shape

((583, 26), (122, 19))

In [ ]:
#выгружаем в файл для повторного использования без обращения к вк
df_u.to_csv('c:/data_all_union.csv',sep = ',', encoding='utf-8')
df_f.to_csv('c:/data_friends.csv',sep = ',', encoding='utf-8')

###  Загрузка данных из файла(без подключения к ВК)


In [ ]:
df_f = pd.read_csv('c:/data_friends.csv',  sep=',')
df_u = pd.read_csv('c:/data_all_union.csv',  sep=',')

### Обработка текста

In [4]:
text = df_u['text']

In [5]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string


stop = set(stopwords.words('russian'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()


def clean(doc):
    #print(doc)
    doc = str(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join([ch for ch in stop_free if ch not in exclude])
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized




In [6]:
doc_clean = [clean(doc) for doc in df_u['text']]   
print(doc_clean[149])

новый год встретили выходя такси положенно проведём дороге откладывая начнём прямо сейчас москвапекинфилиппины 🥳 погнали


In [69]:
#import gensim
#from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
#dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
#doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(doc_clean).todense()
features

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
y = df_u['age3']
X = features

Попробуем линейную регрессию как baseline


In [10]:
from sklearn import cross_validation, tree, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
regr = linear_model.LinearRegression()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [171]:
#df_u['tokens'] = df_u['text'].apply(lambda x: clean(x).split())


In [172]:
#df_u['vector'] = df_u['tokens'].apply(lambda x: dictionary.doc2bow(x))

In [ ]:
#cols = ['id', 'city.id','country.id','relation','sex','graduation']
#X = df_u[cols]
#y = df_u['age3']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [178]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f " % mse)

MSE: 1568606792827209747791872.00 


In [15]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f " % mse)

MSE: 381.95 


In [16]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0)
clf.fit(X_train, y_train) 

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [17]:
y_pred = clf.predict(X_test)

In [18]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f " % mse)

MSE: 456.25 


In [25]:
def randomized_cv(model, param_grid, x_train=X_train, y_train=y_train):
    grid_search = RandomizedSearchCV(model, param_grid, cv=5, scoring='mean_squared_error')
    t_start = time.time()
    grid_search.fit(x_train, y_train)
    t_end = time.time()
    print('model {} best rmsle is {}'.format(model.__class__.__name__, grid_search.best_score_))
    print('time for training is {} seconds'.format(t_end - t_start))
    print(grid_search.best_score_)
    return grid_search.best_estimator_

In [26]:
import xgboost as xgb

param_grid = {
    'max_depth': [2, 3, 4],
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.025]
}
xgb = randomized_cv(xgb.XGBRegressor(), param_grid)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\ProgramData\Anaconda3\lib\sit

model XGBRegressor best rmsle is -379.5149281501976
time for training is 322.83346486091614 seconds
-379.5149281501976


In [27]:
xgb

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.025, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [28]:
y_pred = xgb.predict(X_test) 

In [29]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f " % mse)

MSE: 450.06 
